In [1]:
import sys
sys.path.append("../../smsl")

In [2]:
from smsl.opt import rcsb2charmmformat_sh, charmm2rcsbformat_sh, charmm_c43b1, toppar

### Config Setting

In [3]:
pdbid = "dsdna"

In [4]:
pymol = "/opt/PyMOL-3.0.3_appveyor849-Linux-x86_64-py310/pymol/bin/pymol"

In [5]:
print(f'''\
rcsb2charmmformat_sh = {rcsb2charmmformat_sh}
charmm2rcsbformat_sh = {charmm2rcsbformat_sh}
charmm_c43b1 = {charmm_c43b1}
pymol = {pymol}
''')

rcsb2charmmformat_sh = /home/user/workfolder/g4dna_protocol/smsl/opt/atomname_transfer/rcsb2charmmformat.sh
charmm2rcsbformat_sh = /home/user/workfolder/g4dna_protocol/smsl/opt/atomname_transfer/charmm2rcsbformat.sh
charmm_c43b1 = /home/user/workfolder/g4dna_protocol/smsl/opt/c43b1/bin/charmm
pymol = /opt/PyMOL-3.0.3_appveyor849-Linux-x86_64-py310/pymol/bin/pymol



### 1.Download fd_helix.c and compile

In [6]:
## 因為無法驗證該網站的 HTTPS 憑證，--no-check-certificate 用來繞過該憑證以下載
!wget --no-check-certificate https://casegroup.rutgers.edu/fd_helix.c

--2024-12-18 11:09:17--  https://casegroup.rutgers.edu/fd_helix.c
Resolving casegroup.rutgers.edu (casegroup.rutgers.edu)... 128.6.241.202
Connecting to casegroup.rutgers.edu (casegroup.rutgers.edu)|128.6.241.202|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 26841 (26K) [text/plain]
Saving to: ‘fd_helix.c’

fd_helix.c          100%[===================>]  26.21K   121KB/s    in 0.2s    

2024-12-18 11:09:18 (121 KB/s) - ‘fd_helix.c’ saved [26841/26841]



In [7]:
## -lm links the math library, which is necessary for mathematical functions used in the code.
!gcc -o fd_helix fd_helix.c -lm

In [8]:
!./fd_helix abdna AGGGTTAGGGTTAGGGTTAGGG > {pdbid.upper()}.pdb

### 2. Preprocess PDB File

In [9]:
pymol_script = f'''\
## 1. load pdb file from rcsb
load {pdbid.upper()}.pdb
## 2. modify dna resid and chain
alter resid  1-22, chain='A'
alter resid 23-44, chain='B'
alter resid 23-44, resi=str(int(resi)-22)
## 3. (optional) optimize the 3D view
set sphere_scale, 0.6
zoom
## 4. save the molecule of dna and ion part separately to pdb
save {pdbid}_chain_a.pdb, chain 'A'
save {pdbid}_chain_b.pdb, chain 'B'
save aa.pdb
'''
!echo "{pymol_script}" > {pdbid}_setup.pml
!{pymol} -c {pdbid}_setup.pml

 PyMOL(TM) 3.0.3 - Incentive Product
 Copyright (C) Schrodinger, LLC
 
 This Executable Build integrates and extends Open-Source PyMOL.
 Detected 12 CPU cores.  Enabled multithreaded rendering.
PyMOL>load DSDNA.pdb
 CmdLoad: "" loaded as "DSDNA".
PyMOL>alter resid  1-22, chain='A'
 Alter: modified 465 atoms.
PyMOL>alter resid 23-44, chain='B'
 Alter: modified 431 atoms.
PyMOL>alter resid 23-44, resi=str(int(resi)-22)
 Alter: modified 431 atoms.
PyMOL>set sphere_scale, 0.6
 Setting: sphere_scale set to 0.60000.
PyMOL>zoom
PyMOL>save dsdna_chain_a.pdb, chain 'A'
 Save: wrote "dsdna_chain_a.pdb".
PyMOL>save dsdna_chain_b.pdb, chain 'B'
 Save: wrote "dsdna_chain_b.pdb".
PyMOL>save aa.pdb
 Save: wrote "aa.pdb".


## 3. Transfer rcsb format to charmm format

In [10]:
## transfer to charmm format
!bash {rcsb2charmmformat_sh} {pdbid}_chain_a.pdb > {pdbid}_chain_a.0.pdb
!bash {rcsb2charmmformat_sh} {pdbid}_chain_b.pdb > {pdbid}_chain_b.0.pdb

## 4. Setup Dna via charmm

In [11]:
charmm_script = f'''\
* Author: Shih-Hsueh Hsu (Allen)
* Title: Generate the Double Strand DNA sturcture 
*        and export to .pdb file
*

set curdir = .
set topdir = {toppar}
set pdbdir = .


set dna = {pdbid}
set a_part = chain_a
set b_part = chain_b

open unit 11 read form name @topdir/top_all36_na.rtf
read rtf card unit 11
close unit 11

open read unit 11 card name @topdir/par_all36_na.prm
read param unit 11 flex
close unit 11

open read unit 11 card name @topdir/toppar_water_ions.str
stream card unit 11
close unit 11

read sequ card
*  
   22
!AGGGTTAGGGTTAGGGTTAGGG
ADE GUA GUA GUA THY THY ADE GUA GUA GUA 
THY THY ADE GUA GUA GUA THY THY ADE GUA 
GUA GUA  
generate A setup first 5ter last 3ter

! (default)RNA -> DNA 
patch deo5 A  1 setup warn
patch deox A  2 setup warn
patch deox A  3 setup warn
patch deox A  4 setup warn
patch deox A  5 setup warn
patch deox A  6 setup warn
patch deox A  7 setup warn
patch deox A  8 setup warn
patch deox A  9 setup warn
patch deox A 10 setup warn
patch deox A 11 setup warn
patch deox A 12 setup warn
patch deox A 13 setup warn
patch deox A 14 setup warn
patch deox A 15 setup warn
patch deox A 16 setup warn
patch deox A 17 setup warn
patch deox A 18 setup warn
patch deox A 19 setup warn
patch deox A 20 setup warn
patch deox A 21 setup warn
patch deox A 22 setup warn 

read sequ card
*  
   22
!AGGGTTAGGGTTAGGGTTAGGG
CYT CYT 
CYT THY ADE ADE CYT CYT CYT THY ADE ADE 
CYT CYT CYT THY ADE ADE CYT CYT CYT THY 

generate B setup first 5ter last 3ter

! (default)RNA -> DNA 
patch deo5 B  1 setup warn
patch deox B  2 setup warn
patch deox B  3 setup warn
patch deox B  4 setup warn
patch deox B  5 setup warn
patch deox B  6 setup warn
patch deox B  7 setup warn
patch deox B  8 setup warn
patch deox B  9 setup warn
patch deox B 10 setup warn
patch deox B 11 setup warn
patch deox B 12 setup warn
patch deox B 13 setup warn
patch deox B 14 setup warn
patch deox B 15 setup warn
patch deox B 16 setup warn
patch deox B 17 setup warn
patch deox B 18 setup warn
patch deox B 19 setup warn
patch deox B 20 setup warn
patch deox B 21 setup warn
patch deox B 22 setup warn 

read coor pdb name @curdir/@dna_@{{a_part}}.0.pdb offset +0
read coor pdb name @curdir/@dna_@{{b_part}}.0.pdb offset +22
print coor

define fix select initialized .and. .not. hydrogen show end

define nonH select .not. hydrogen end
set nonHatoms = ?nsel
define missing select .not. hydrogen .and. .not. initialized show end
set missingatoms = ?nsel

Calc fraction = (@nonHatoms - @missingatoms) / @nonHatoms


! Use of AUTOGENERATE is essential
autogenerate angles dihedrals 

! Internal Coordinate Manipulation
ic fill preserve  !fill: autocomplete hydrogen
ic param
ic build

scaler wmain set 0.0 sele all end
scaler wmain set 1.0 sele fix end

cons fix sele fix end

mini sd   nstep 100
mini abnr nstep 1000
mini abnr nstep 5000

open write unit 12 card name @curdir/@{{dna}}.aa.pdb
write coor pdb unit 12
close unit 12

stop


'''
!echo "{charmm_script}" > {pdbid}.inp

In [12]:
!{charmm_c43b1} < {pdbid}.inp > {pdbid}.dat

### 5. Transfer charmm format to rcsb format

In [13]:
!bash {charmm2rcsbformat_sh} {pdbid}.aa.pdb > {pdbid}.pgro.pdb

In [14]:
## Add TER between chain A and chain B
import subprocess

command = f"""\
sed -i '/ A /{{
    n;/ B /i TER
}}' {pdbid}.pgro.pdb
"""
subprocess.run(command, shell=True, check=True)

CompletedProcess(args="sed -i '/ A /{\n    n;/ B /i TER\n}' dsdna.pgro.pdb\n", returncode=0)

### (optinal) 6. Visulize via PyMol

In [15]:
!{pymol} {pdbid}.pgro.pdb

 PyMOL(TM) 3.0.3 - Incentive Product
 Copyright (C) Schrodinger, LLC
 
 This Executable Build integrates and extends Open-Source PyMOL.
 License Expiry date: 01-mar-2025
 Detected OpenGL version 4.6. Shaders available.
 Detected GLSL version 4.60.
 OpenGL graphics engine:
  GL_VENDOR:   NVIDIA Corporation
  GL_RENDERER: NVIDIA GeForce GTX 1650/PCIe/SSE2
  GL_VERSION:  4.6.0 NVIDIA 535.183.01
 Detected 12 CPU cores.  Enabled multithreaded rendering.
 CmdLoad: "" loaded as "dsdna.pgro".
